# fastAPI pipeline

__Goal__: 


### Import

In [1]:
%load_ext autoreload
%autoreload 2
import joblib
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')
from pathlib import Path
import os
from pydantic import BaseModel


C:\Users\satyr\AppData\Local\Temp\ipykernel_36084\2998831471.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data_dir =  Path.cwd().parent / "data"
models_dir = Path.cwd().parent / "models"
models_dir.mkdir(exist_ok=True)

 ### Utilities

In [3]:
def update_current_month_df(current_month_df, new_item_df):
    return pd.concat([current_month_df, new_item_df], axis=0)

def update_current_pred(current_month_predictions_sr, new_predictions_sr):
    return pd.concat([current_month_predictions_sr, new_predictions_sr], axis=0)


def pop_first_row(df:pd.DataFrame):
    if not df.empty:
        first_row = df.iloc[0].copy()
        df.drop(0, inplace=True)
        df.reset_index(inplace=True, drop=True)
        return first_row
    else:
        # Handle the case when the DataFrame is empty, e.g., return None or raise an exception.
        return None

def mimic_api_call(production_data_df):
    row = pop_first_row(production_data_df)
    row = row.fillna('')
    item_data = Item(**row.to_dict())
    sent_json = item_data.dict()
    return sent_json

def json_to_item_df(received_json):
    item = Item(**received_json)
    item_df = pd.DataFrame([item.dict()])
    return item_df

def predict(previous_item_df, new_item_df):
    df =  pd.concat([previous_item_df, new_item_df], axis=0)
    result_data = data_ingestion_pipeline.transform(df)
    y = model.predict(predictors_feature_eng_transformer.transform(result_data))
    return y[-1]


class Item(BaseModel):
    S_No:int
    Timestamp: str = ''
    Location: str
    Temperature_C: float = None
    Apparent_Temperature_C: float = None
    Humidity: float = None
    Wind_speed_kmph: float = None
    Wind_bearing_degrees: float = None
    Visibility_km: float = None
    Pressure_millibars: float = None
    Weather_conditions: str = ''

In [4]:




model_folder = models_dir

model = joblib.load(model_folder / 'model.pkl')
predictors_feature_eng_transformer = joblib.load(model_folder / 'predictors_feature_eng_pipeline.pkl')
data_ingestion_pipeline = joblib.load(model_folder / 'dataset_ingestion_pipeline.pkl')



# Main

In [5]:
# Define the Pydantic model for the JSON data
production_data_df = pd.read_csv(data_dir / 'weather_dataset_raw_production.csv')
production_data_df = production_data_df.head(200)
previous_item_df = pd.read_csv(data_dir / 'weather_dataset_raw_development.csv')[-1:]



In [6]:
current_month_df = None
current_month_predictions_sr = None

for i in range(200):
    received_json = mimic_api_call(production_data_df)
    new_item_df = json_to_item_df(received_json)
    current_month_df = update_current_month_df(current_month_df, new_item_df)
    y = predict(previous_item_df, new_item_df)
    current_month_predictions_sr = update_current_pred(current_month_predictions_sr, pd.Series([y]) )
    previous_item_df = new_item_df
current_month_df


,S_No,Timestamp,Location,Temperature_C,Apparent_Temperature_C,Humidity,Wind_speed_kmph,Wind_bearing_degrees,Visibility_km,Pressure_millibars,Weather_conditions
0,46729,2011-01-01 00:00:00+00:00,"Port of Turku, Finland",-7.100000,-7.100000,0.96,3.8962,195.0,3.9123,1025.25,snow
0,46730,2011-01-01 01:00:00+00:00,"Port of Turku, Finland",-7.061111,-7.061111,1.00,3.2039,171.0,3.4132,1025.07,snow
0,46731,2011-01-01 02:00:00+00:00,"Port of Turku, Finland",-6.550000,-6.550000,0.96,3.2039,161.0,2.4955,1025.14,snow
0,46732,2011-01-01 03:00:00+00:00,"Port of Turku, Finland",-6.538889,-9.705556,0.96,6.3917,185.0,2.6082,1024.62,snow
0,46733,2011-01-01 04:00:00+00:00,"Port of Turku, Finland",-6.038889,-10.788889,0.93,10.8997,190.0,2.9624,1024.26,snow
...,...,...,...,...,...,...,...,...,...,...,...
0,47452,2011-01-09 03:00:00+00:00,"Port of Turku, Finland",3.322222,0.616667,0.93,10.2235,203.0,11.3022,1020.67,rain
0,47453,2011-01-09 04:00:00+00:00,"Port of Turku, Finland",2.588889,-0.283333,1.00,10.3362,189.0,9.5312,1020.50,rain
0,47454,2011-01-09 05:00:00+00:00,"Port of Turku, Finland",2.522222,-0.005556,0.96,8.8872,188.0,9.5312,1020.78,rain
0,47455,2011-01-09 06:00:00+00:00,"Port of Turku, Finland",1.977778,-0.494444,1.00,8.3398,153.0,4.4436,1021.01,rain


In [7]:
current_month_predictions_sr

0    0.0
0    0.0
0    0.0
0    0.0
0    0.0
    ... 
0    1.0
0    1.0
0    1.0
0    1.0
0    1.0
Length: 200, dtype: float64